In [3]:
# 1. ENVIRONMENT STABILIZATION
# Upgrading to ensure 2026-standard ABI 15 and 4-bit support
!pip install -U -q bitsandbytes transformers accelerate tree-sitter
!rm -rf tree-sitter-tcl
!git clone https://github.com/tree-sitter-grammars/tree-sitter-tcl -q

import os, ctypes, torch, torch.nn as nn
from tree_sitter import Language, Parser
from transformers import LlamaForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch.amp import autocast, GradScaler

# 2. COMPILE & BIND PARSER (ABI 15 COMPATIBLE)
!mkdir -p build
!gcc -shared -o build/tcl.so -fPIC -I./tree-sitter-tcl/src \
  ./tree-sitter-tcl/src/parser.c ./tree-sitter-tcl/src/scanner.c

lib = ctypes.cdll.LoadLibrary('build/tcl.so')
tcl_func = getattr(lib, "tree_sitter_tcl")
tcl_func.restype = ctypes.c_void_p
ptr = tcl_func()

# SMART INITIALIZATION: Detects 2024 (1 arg) vs 2026 (2 args) API
try:
    TCL_LANGUAGE = Language(ptr, "tcl")
    parser = Parser(TCL_LANGUAGE)
    print("✅ Parser: Using modern API (0.22+)")
except TypeError:
    TCL_LANGUAGE = Language(ptr)
    parser = Parser(TCL_LANGUAGE)
    print("✅ Parser: Using stable API (0.21.x)")

# 3. DEFINE ARCHITECTURE (Additive Structural Injection)
class ASTEmbeddingLayer(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.type_embed = nn.Embedding(256, hidden_size)
        self.depth_embed = nn.Embedding(64, hidden_size)
        self.sibling_embed = nn.Embedding(128, hidden_size)
        self.proj = nn.Linear(hidden_size, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self, t, d, s):
        # Kept in float32 for training stability
        return self.layer_norm(self.proj(
            self.type_embed(t) + self.depth_embed(d) + self.sibling_embed(s)
        ))

class ASTAugmentedLlama(nn.Module):
    def __init__(self, model_id="codellama/CodeLlama-7b-hf"):
        super().__init__()
        self.base_model = LlamaForCausalLM.from_pretrained(
            model_id, 
            quantization_config=BitsAndBytesConfig(load_in_4bit=True), 
            device_map="auto"
        )
        self.ast_injector = ASTEmbeddingLayer(self.base_model.config.hidden_size).to("cuda")

    def forward(self, input_ids, node_types, depths, siblings, labels=None):
        embeds = self.base_model.get_input_embeddings()(input_ids)
        ast_signal = self.ast_injector(node_types, depths, siblings)
        # Injection: e_final = e_token + e_AST
        # Cast signal to float16 to match Llama layers and avoid RuntimeError
        return self.base_model(inputs_embeds=embeds + ast_signal.to(embeds.dtype), labels=labels)

# 4. DATA ENGINE (Character-to-AST Mapping)
def get_metadata(code, tokenizer):
    tree = parser.parse(bytes(code, "utf8"))
    enc = tokenizer(code, truncation=True, max_length=512, return_offsets_mapping=True, padding="max_length")
    node_types, depths, siblings = [], [], []
    for start, end in enc['offset_mapping']:
        if start == end: 
            node_types.append(0); depths.append(0); siblings.append(0); continue
        node = tree.root_node.descendant_for_byte_range(start, end)
        node_types.append(hash(node.type) % 256)
        d, p = 0, node
        while p.parent: d += 1; p = p.parent
        depths.append(min(d, 63))
        siblings.append(min(node.child_index if hasattr(node, 'child_index') else 0, 127))
    return {k: torch.tensor([v]).to("cuda") for k, v in {"input_ids": enc.input_ids, "node_types": node_types, "depths": depths, "siblings": siblings}.items()}

# 5. EXECUTION & ALIGNMENT TRAINING
print("🚀 Loading Model...")
model_id = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = ASTAugmentedLlama(model_id)

target_script = 'proc factorial {n} { if {$n <= 1} { return 1 } { return [expr {$n * [factorial [expr {$n-1}]]}] } }'
inputs = get_metadata(target_script, tokenizer)

# Dynamic Baseline Calculation (Zeroing AST)
model.eval()
with torch.no_grad():
    zero_signal = {k: torch.zeros_like(v) if k != "input_ids" else v for k, v in inputs.items()}
    baseline_loss = model(**zero_signal, labels=inputs["input_ids"]).loss.item()

# Alignment Loop
optimizer = torch.optim.AdamW(model.ast_injector.parameters(), lr=1e-4)
scaler = GradScaler('cuda')
model.train()
print(f"✨ Training Structural Alignment (Dynamic Baseline: {baseline_loss:.4f})")

for step in range(101):
    optimizer.zero_grad()
    with autocast('cuda'):
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    if step % 50 == 0: print(f"Step {step:03} | Alignment Loss: {loss.item():.4f}")

# 6. FINAL RESULTS
model.eval()
with torch.no_grad():
    final_loss = model(**inputs, labels=inputs["input_ids"]).loss.item()

print("\n--- FINAL QUANTITATIVE RESULTS ---")
print(f"Baseline Loss (No AST): {baseline_loss:.4f}")
print(f"Augmented Loss (Aligned): {final_loss:.4f}")
improvement = ((baseline_loss - final_loss) / baseline_loss) * 100
print(f"📈 Net Improvement: {improvement:.2f}%")

2026-01-15 07:39:03.401617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768462743.849667      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768462743.970552      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768462745.026872      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768462745.026908      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768462745.026911      55 computation_placer.cc:177] computation placer alr

✅ Parser: Using stable API (0.21.x)
🚀 Loading Model...


/tmp/ipykernel_55/1402359989.py:27: DeprecationWarning: int argument support is deprecated
  TCL_LANGUAGE = Language(ptr)


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✨ Training Structural Alignment (Dynamic Baseline: 7.3645)
Step 000 | Alignment Loss: 7.3968
Step 050 | Alignment Loss: 0.7423
Step 100 | Alignment Loss: 0.4798

--- FINAL QUANTITATIVE RESULTS ---
Baseline Loss (No AST): 7.3645
Augmented Loss (Aligned): 0.4755
📈 Net Improvement: 93.54%


In [4]:
# 1. DEFINE VALIDATION SUITE
validation_suite = {
    "Recursive Factorial": "proc fact {n} { if {$n <= 1} { return 1 } { return [expr {$n * [fact [expr {$n-1}]]}] } }",
    "Conditional Logic": "proc check {val} { set status 0; if {$val > 10} { set status 1 } else { set status -1 }; return $status }",
    "List Manipulation": "proc process_list {items} { set result {}; foreach item $items { lappend result [string toupper $item] }; return $result }"
}

results = []

# 2. RUN EVALUATION
model.eval()
print(f"{'Scenario':<25} | {'Baseline':<10} | {'Augmented':<10} | {'Gain (%)':<10}")
print("-" * 65)

for name, code in validation_suite.items():
    inputs = get_metadata(code, tokenizer) # Using your existing helper
    
    with torch.no_grad():
        with autocast('cuda'):
            # Calculate Baseline (Zeroed AST)
            zero_signal = {k: torch.zeros_like(v) if k != "input_ids" else v for k, v in inputs.items()}
            baseline = model(**zero_signal, labels=inputs["input_ids"]).loss.item()
            
            # Calculate Augmented (Aligned)
            augmented = model(**inputs, labels=inputs["input_ids"]).loss.item()
            
    gain = ((baseline - augmented) / baseline) * 100
    results.append({"Scenario": name, "Baseline": baseline, "Augmented": augmented, "Gain": gain})
    print(f"{name:<25} | {baseline:<10.4f} | {augmented:<10.4f} | {gain:>8.2f}%")

avg_gain = sum(r['Gain'] for r in results) / len(results)
print("-" * 65)
print(f"AVERAGE STRUCTURAL GAIN: {avg_gain:.2f}%")

Scenario                  | Baseline   | Augmented  | Gain (%)  
-----------------------------------------------------------------
Recursive Factorial       | 0.9266     | 0.4537     |    51.04%
Conditional Logic         | 0.8793     | 0.5145     |    41.49%
List Manipulation         | 0.8964     | 0.5979     |    33.30%
-----------------------------------------------------------------
AVERAGE STRUCTURAL GAIN: 41.94%


In [5]:
# ABLATION EXPERIMENT
ablation_scenarios = {
    "Baseline (Token Only)": {"t": False, "d": False, "s": False},
    "Token + Node Type": {"t": True, "d": False, "s": False},
    "Token + Type + Depth": {"t": True, "d": True, "s": False},
    "Full (Type + Depth + Sibling)": {"t": True, "d": True, "s": True}
}

print(f"{'Configuration':<30} | {'NLL Loss':<10} | {'Reduction (%)':<10}")
print("-" * 60)

for label, config in ablation_scenarios.items():
    # Helper to mask features by zeroing them out
    mask_inputs = inputs.copy()
    if not config["t"]: mask_inputs["node_types"] = torch.zeros_like(inputs["node_types"])
    if not config["d"]: mask_inputs["depths"] = torch.zeros_like(inputs["depths"])
    if not config["s"]: mask_inputs["siblings"] = torch.zeros_like(inputs["siblings"])
    
    with torch.no_grad():
        with autocast('cuda'):
            loss = model(**mask_inputs, labels=inputs["input_ids"]).loss.item()
    
    reduction = ((baseline_loss - loss) / baseline_loss) * 100
    print(f"{label:<30} | {loss:<10.4f} | {reduction:>8.2f}%")

Configuration                  | NLL Loss   | Reduction (%)
------------------------------------------------------------
Baseline (Token Only)          | 0.8964     |    87.83%
Token + Node Type              | 0.7800     |    89.41%
Token + Type + Depth           | 0.5979     |    91.88%
Full (Type + Depth + Sibling)  | 0.5979     |    91.88%


In [11]:
import torch.nn.functional as F
import numpy as np

# 1. RUN INFERENCE
model.eval()
with torch.no_grad():
    with torch.amp.autocast('cuda'):
        # Baseline: Structural side-car zeroed
        zero_out = {k: torch.zeros_like(v) if k not in ["input_ids"] else v 
                    for k, v in inputs.items()}
        base_logits = model(**zero_out).logits
        
        # Augmented: Aligned structural signal
        aug_logits = model(**inputs).logits

# 2. ALIGN FOR NEXT-TOKEN PREDICTION (The Shift)
# Logits at [i] predict Token at [i+1]
labels = inputs["input_ids"][:, 1:].contiguous()       # Targets
logits_aug = aug_logits[:, :-1, :].transpose(1, 2)    # Predictions
logits_base = base_logits[:, :-1, :].transpose(1, 2)

# 3. CALCULATE LOSS DELTA
base_loss = F.cross_entropy(logits_base, labels, reduction='none')
aug_loss = F.cross_entropy(logits_aug, labels, reduction='none')
diff = (base_loss - aug_loss).cpu().numpy()[0]

# 4. MANUAL MASKING (Filtering out </s> tokens)
# We find where the target IDs are NOT the EOS token (usually ID 2)
target_ids = labels[0].cpu().numpy()
eos_token_id = tokenizer.eos_token_id or 2 
valid_indices = np.where(target_ids != eos_token_id)[0]

# Extract actual code tokens and their gains
valid_tokens = [tokenizer.convert_ids_to_tokens([target_ids[i]])[0] for i in valid_indices]
valid_diffs = diff[valid_indices]

# 5. DISPLAY RESULTS (The Heatmap Data)
print(f"{'Index':<6} | {'Token':<15} | {'Confidence Gain (Loss Delta)':<25}")
print("-" * 55)
for i in range(min(30, len(valid_tokens))):
    t_clean = valid_tokens[i].replace(' ', ' ')
    print(f"{i:<6} | {t_clean:<15} | {valid_diffs[i]:>20.4f}")

# 6. AGGREGATED GAINS BY SYNTAX (Final Table 6 Data)
syntax_groups = {
    "Control Flow (proc, if)": [],
    "Structural Grouping ({ }, [ ])": [],
    "Logic & Operations (expr, set)": []
}

for t, d in zip(valid_tokens, valid_diffs):
    t_str = t.replace(' ', '').strip()
    if t_str in ['proc', 'if', 'else', 'return', 'foreach']:
        syntax_groups["Control Flow (proc, if)"].append(d)
    elif t_str in ['{', '}', '[', ']']:
        syntax_groups["Structural Grouping ({ }, [ ])"].append(d)
    elif any(op in t_str for op in ['expr', 'set', '<=', '==', '*', '+']):
        syntax_groups["Logic & Operations (expr, set)"].append(d)

print("\n--- AGGREGATED GAINS FOR TABLE 6 ---")
for cat, gains in syntax_groups.items():
    if gains:
        print(f"{cat:<30}: Mean Delta = {sum(gains)/len(gains):.4f}")

Index  | Token           | Confidence Gain (Loss Delta)
-------------------------------------------------------
0      | <s>             |               0.0000
1      | ▁proc           |               0.0000
2      | ▁process        |               4.2500
3      | _               |               3.8281
4      | list            |               3.6328
5      | ▁{              |               6.7617
6      | items           |               4.7891
7      | }               |               6.3242
8      | ▁{              |               7.0898
9      | ▁set            |               4.7656
10     | ▁result         |               4.7500
11     | ▁{};            |               5.2266
12     | ▁foreach        |               4.5547
13     | ▁item           |               4.6250
14     | ▁$              |               5.2930
15     | items           |               4.5234
16     | ▁{              |               6.5977
17     | ▁la             |               4.0703
18     | pp             